In [1]:
# BE CAREFUL, this will cause a version change in your system, use in a virtual environment
%pip install networkx==2.8.7

  Using cached networkx-2.8.7-py3-none-any.whl (2.0 MB)
  Attempting uninstall: networkx
    Found existing installation: networkx 3.1
    Uninstalling networkx-3.1:
      Successfully uninstalled networkx-3.1
Note: you may need to restart the kernel to use updated packages.


In [12]:
import sys
sys.path.append("/Users/miracle/Desktop/safepy")
from os.path import expanduser

# Add path to folder containing safepy
sys.path.append(expanduser('~') + '/Lab/Utils/Python/Networks/safepy/')

import safe
from goatools import obo_parser
import pandas as pd
import matplotlib as plt

%matplotlib inline

Use this command to create annotations:

python3.8 safepy/utils/make_go.py --path-to-obo ../Downloads/go.obo --path-to-annotations ../Downloads/wb.gaf.gz

The files are taken from here:

obo - http://geneontology.org/docs/download-ontology/

annotations - http://current.geneontology.org/products/pages/downloads.html

In [ ]:
networks = ['yeast_netwoks.txt', 'yeast_gm_netwoks.txt', 'yeast_filter_networks.txt',
            'human_netwoks.txt', 'human_another_netwoks.txt']
annotate_matrix = ['yeast_annotate_matrix.txt', 'yeast_annotate_matrix.txt', 'yeast_annotate_matrix.txt',
                   'human_annotate_matrix.txt', 'human_annotate_matrix.txt']

In [ ]:
# The same operations are performed for each matrix, so a loop is used to avoid code duplication

sfs = []
go = obo_parser.GODag('../../Downloads/go-basic.obo')

for i in range(len(networks)):
    
    sf = safe.SAFE() #create safe object
    sf.load_network(network_file=networks[i]) #read network
    sf.load_attributes(attribute_file=annotate_matrix[i]) #download annotation
    #find neighborhoods 
    sf.define_neighborhoods(node_distance_metric='shortpath_weighted_layout', neighborhood_radius=0.15)
    sf.compute_pvalues(multiple_testing=False)
    
    #change GO names to id (without 'GO:')
    for go_id in sf.attributes.name:
        go_term = go_id.split(':')[1]
        sf.attributes.name.replace(go_id, go_term, inplace=True)
    
    #clustering
    sf.define_top_attributes()
    sf.define_domains(attribute_distance_threshold = 0.65)
    sf.trim_domains()
    
    #change GO id to description
    for cluster in sf.domains.label.items():
        terms = ''
    for GO_number in cluster[1].split(', '):
        term = go['GO:'+GO_number].name
        terms+='\n'+term
    sf.domains.label.replace(cluster, str(terms), inplace=True)
    
    sfs.append(sf)

### You can refer to different objects by the index

In [ ]:
#plot of network
sfs[4].plot_network()

In [ ]:
#this is how you can save pictures with clusters
sf[1].plot_composite_network(show_each_domain=True,
                         save_fig='aaaa.svg')